In [0]:
import pickle
import cv2
import keras
import matplotlib.pyplot as plt
import numpy as np
import csv
import random
import os
from skimage.transform import resize
from keras.datasets import mnist
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.layers.core import Dropout
from keras import regularizers
from keras.models import *
from keras.preprocessing.image import ImageDataGenerator
import time
from keras import backend as K
import tensorflow as tf
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model
from keras import regularizers
from keras.callbacks import ModelCheckpoint
import skimage.io as io
import skimage.transform as trans
from keras.layers import *
from keras.optimizers import *

random.seed(42)

if not os.path.exists("figures"):
  os.mkdir("figures")

class LossLog(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_begin(self,epoch,logs={}):
        self.losses_epoch = []

    def on_epoch_end(self,epoch,logs={}):
        self.losses.append(self.losses_epoch)

    def on_batch_end(self, batch, logs={}):
        self.losses_epoch.append(logs.get('loss'))

    def on_train_end(self,logs={}):
        layers=len(self.model.layers)
        timestr = time.strftime("%Y%m%d-%H%M%S")
        save_loss_to_csv(self.losses,layers,timestr)

def save_pickle(path, obj):
    print('Saving object in: ' + path)
    with open(path, 'wb+') as f:
        pickle.dump(obj, f)

def load_pickle(path):
    print('Loading object from: ' + path)
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_model(model,filename):
    model.save("/content/"+filename+".h5")    

def display_images(original,reconstructed):
    res=np.concatenate((original,reconstructed),axis=1)
    plt.imshow(res)
    plt.show()
    
def display_images3(original,y_value,reconstructed, name_model):
    f = plt.figure()
    f.add_subplot(1,3, 1)
    plt.imshow(original)
    plt.title("Original")
    f.add_subplot(1,3, 2)
    plt.imshow(y_value)
    plt.title("Segmentation")
    f.add_subplot(1,3, 3)
    plt.imshow(reconstructed)
    plt.title("Output")
    plt.show(block=True)
    f.savefig("figures/"+name_model+".png")
    
#im_nums: IDs of the images
#x_val: Dataset of the original images of validation (RGB)
#gray_x_val: Dataset of the original images of validation (grayscale)
#y_val: Dataset of the y-values of validation
#model: NN that is being analyzed 
#name_model: name of the file in which it will be stored
def displayAndSaveImages3(im_nums,x_val,gray_x_val,y_val,model,name_model):
    im_iter = 0
    rows = len(im_nums)
    f = plt.figure(figsize=(10,int(15*rows/4)))
    #f=plt.figure()
    for i in range(len(im_nums)):
      img_x = gray_x_val[im_nums[i]:(im_nums[i]+1)]
      img_y = y_val[im_nums[i]:(im_nums[i]+1)]
      img = model.predict(img_x)
      original = x_val[im_nums[i]:(im_nums[i]+1)][0]
      segmentation = img_y[0][:,:,0]
      reconstructed = img[0][:,:,0]      
      f.add_subplot(rows,3, int(1 + im_iter*3))
      plt.imshow(original)
      plt.title("Original")
      f.add_subplot(rows,3, int(2 + im_iter*3))
      plt.imshow(segmentation)
      plt.title("Segmentation")
      f.add_subplot(rows,3, int(3 + im_iter*3))
      plt.imshow(reconstructed)      
      plt.title("Output")
      im_iter = im_iter + 1
    f.savefig("figures/"+name_model+"_testFigures.png")
    
    
def save_loss_to_csv(losses,layers,timestr):
    filepath="/content/"+timestr+"_layers-"+str(layers)+".csv"

    with open(filepath, 'w+') as csvFile:
        writer = csv.writer(csvFile)
        for epoch in losses:
            writer.writerow(epoch)

    csvFile.close()
    
    
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

  
def dice_coef_loss(y_true, y_pred):
  return -dice_coef(y_true, y_pred)


def dice_loss(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  # some implementations don't square y_pred
  denominator = tf.reduce_sum(y_true + tf.square(y_pred))
  return -numerator/(denominator + tf.keras.backend.epsilon())


def binary_crossentropy(y_true, y_pred):
  return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)


def loss_funct(y_true, y_pred):
  return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))


def RGB2GRAY(x):
  grayImages = np.zeros((x.shape[0],x.shape[1],x.shape[1],1))
  for i in range(len(x)):
    grayImages[i,:,:,0] = cv2.cvtColor(x[i], cv2.COLOR_RGB2GRAY)
  return grayImages

image_size = 128



Using TensorFlow backend.


     |████████████████████████████████| 993kB 4.9MB/s 


### ***Load data***

In [0]:
 # Authenticate and create the PyDrive client.
 # This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Segmentation all
train_file_id = '151xqm5vZwFdAZeqnjc0mt1QJEdyuWPMt'
x_train_file = drive.CreateFile({'id': train_file_id})
val_file_id = '1fRLHfjhE4GxTCl4mSeL1e3vFrPwVvO98'
x_val_file = drive.CreateFile({'id': val_file_id})
test_file_id = '1fRLHfjhE4GxTCl4mSeL1e3vFrPwVvO98'
x_test_file = drive.CreateFile({'id': test_file_id})

y_train_file_id = '10heHA2R1ghKTKkd86QJ_JbBA09Baeomw'
y_train_file = drive.CreateFile({'id': y_train_file_id})
y_val_file_id = '1VIR0WwZ7Yx15gXaKhZfHoj1QJrOeWpSO'
y_val_file = drive.CreateFile({'id': y_val_file_id})
y_test_file_id = '1VIR0WwZ7Yx15gXaKhZfHoj1QJrOeWpSO'
y_test_file = drive.CreateFile({'id': y_test_file_id})

x_train_file.GetContentFile('x_train')
x_val_file.GetContentFile('x_val')
x_test_file.GetContentFile('x_test')

y_train_file.GetContentFile('y_train')
y_val_file.GetContentFile('y_val')
y_test_file.GetContentFile('y_test')


x_train=load_pickle('x_train')
y_train=load_pickle("y_train")
x_val=load_pickle('x_val')
y_val=load_pickle("y_val")
x_test=load_pickle('x_test')
y_test=load_pickle("y_test")

gray_x_train = RGB2GRAY(x_train)
gray_x_val = RGB2GRAY(x_val)
gray_x_test = RGB2GRAY(x_test)

x_train_file = 0
x_val_file = 0
x_test_file = 0
y_train_file = 0
y_val_file = 0 
y_test_file = 0 

Loading object from: x_train
Loading object from: y_train
Loading object from: x_val
Loading object from: y_val
Loading object from: x_test
Loading object from: y_test


In [0]:
#%%
#%%  SSSS EEEE GGGG M   M EEEE N  N TTTTT AAAA TTTTT IIIII OOOO N  N                                                                                                  
#%%  S    E    G    MM MM E    NN N   T   A  A   T     I   O  O NN N                                                    
#%%  SSSS EE   G GG M M M EEE  N NN   T   AAAA   T     I   O  O N NN                                                                      
#%%     S E    G  G M   M E    N  N   T   A  A   T     I   O  O N  N                                                                      
#%%  SSSS EEEE GGGG M   M EEEE N  N   T   A  A   T   IIIII OOOO N  N                                                                       
#%%

## ***Whole autoencoder***

### ***Model definition***

In [0]:
#Choose loss function
#lossFunction = 'binary_crossentropy'
lossFunction = dice_coef_loss

segment = Sequential()

#Down
segment.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(image_size, image_size, 1))) 
segment.add(BatchNormalization())
segment.add(MaxPooling2D((2, 2), padding='same'))  #Reduction in size x2.
segment.add(Dropout(0.6))
segment.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
segment.add(BatchNormalization())
segment.add(MaxPooling2D((2, 2), padding='same'))  #Reduction in size x2.
segment.add(Dropout(0.6))
segment.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
segment.add(BatchNormalization())
segment.add(MaxPooling2D((2, 2), padding='same'))  #Reduction in size x2.
segment.add(Dropout(0.6))
#Center
segment.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
segment.add(BatchNormalization())
#Up
segment.add(UpSampling2D(size=(2, 2)))
segment.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
segment.add(BatchNormalization())
segment.add(UpSampling2D(size=(2, 2)))
segment.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
segment.add(BatchNormalization())
segment.add(UpSampling2D(size=(2, 2)))
segment.add(Conv2D(16, (3, 3), activation='relu', padding='same')) 
segment.add(BatchNormalization())
segment.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

segment.summary()
  
segment.compile(optimizer='adam', loss=lossFunction)

callbacks = [
ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

### ***Model training***

In [0]:
#Train the network
history_segment = segment.fit(gray_x_train, y_train, epochs=150, batch_size=16, validation_data=(gray_x_val, y_val), shuffle = True, callbacks = callbacks)
segment.load_weights('model.h5')

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

## ***Save model***

In [0]:
################################################################################
#Parameters to change:
################################################################################
#In name_model put the name of the last executed neural network
name_model = "join_binary_crossentropy"
#Load the model
model = join_model
#Flags
Flag_GreedyLayerWise = 0
Flag_JoinModel = 1
#In history put the name of the history of the NN that you want to save (what is before the fit statement)
history = history_join_model
################################################################################


In [0]:
if not os.path.exists("figures/"+name_model):
  os.mkdir("figures/"+name_model)
  
name_model_sub = name_model+"/"+name_model 
file_name = "figures/" + name_model_sub + "_model.png"
plot_model(model, to_file=file_name, show_shapes=True, show_layer_names=True)


### ***Plot training metrics***

In [0]:
#Plot + save
loss_name = "figures/" + name_model_sub + "_lossFunction.png"

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="r", label="best model")
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val', 'best model'], loc='upper right')
plt.savefig(loss_name)
plt.show()

if Flag_GreedyLayerWise:
  if not os.path.exists("figures/"+name_model+"/submodels"):
     os.mkdir("figures/"+name_model+"/submodels")
  history_models = [history_model1, history_model2, history_model3]
  mod_count = 1
  for hist in history_models:
    history = hist
    #Plot + save
    loss_name = "figures/" + name_model + "/submodels/submodel_" + str(mod_count) + "_lossFunction.png"
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.plot(np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="r", label="best model")
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val', 'best model'], loc='upper right')
    plt.savefig(loss_name)
    plt.show()
    mod_count = mod_count + 1
  


### ***Plot results & save results***

In [0]:
im_nums= [1,2,10,11]

if Flag_JoinModel == 1:
  displayAndSaveImages3(im_nums,x_test,join_x_test,y_test,model,name_model_sub)
else:
  displayAndSaveImages3(im_nums,x_test,gray_x_test,y_test,model,name_model_sub)

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

## ***Greedy layer-wise training***

In [0]:
#Choose loss function
#lossFunction = 'binary_crossentropy'
lossFunction = dice_coef_loss

callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]


#Model with 3 layers

print("Model 1 conv...")
#Model 1
inputs = Input((128, 128, 1))
conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
conv1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
pool1 = Dropout(0.6) (pool1)
up1 = UpSampling2D(size=(2, 2))(pool1) 
up1 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1)
up1 = BatchNormalization()(up1)
up0 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
model1 = Model(inputs=[inputs], outputs=[up0])
model1.summary()
model1.compile(optimizer='adam', loss=lossFunction, metrics=[dice_coef])
history_model1 = model1.fit(gray_x_train, y_train, batch_size=16, nb_epoch=20, verbose=1, shuffle=True, validation_data=(gray_x_val, y_val), callbacks = callbacks)
model1.load_weights('model.h5')
callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]

print("Model 2 conv...")
#Model 2
conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
conv2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
pool2 = Dropout(0.6) (pool2)
up2 = UpSampling2D(size=(2, 2))(pool2) 
up2 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
up2 = BatchNormalization()(up2)
up1 = UpSampling2D(size=(2, 2))(up2) 
up1 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1)
up1 = BatchNormalization()(up1)
up0 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
model2 = Model(inputs=[inputs], outputs=[up0])
model2.layers[1].trainable = False
model2.layers[3].trainable = False
model2.summary()
model2.compile(optimizer='adam', loss=lossFunction, metrics=[dice_coef])
history_model2 = model2.fit(gray_x_train, y_train, batch_size=16, nb_epoch=25, verbose=1, shuffle=True, validation_data=(gray_x_val, y_val), callbacks = callbacks)
model2.load_weights('model.h5')
callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]


print("Model 3 conv...")
#Model 3
conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
conv3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
pool3 = Dropout(0.6) (pool3)
up3 = UpSampling2D(size=(2, 2))(pool3) 
up3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
up3 = BatchNormalization()(up3)
up2 = UpSampling2D(size=(2, 2))(up3) 
up2 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
up2 = BatchNormalization()(up2)
up1 = UpSampling2D(size=(2, 2))(up2) 
up1 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1)
up1 = BatchNormalization()(up1)
up0 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
model3 = Model(inputs=[inputs], outputs=[up0])
model3.layers[1].trainable = False
model3.layers[3].trainable = False
model3.layers[5].trainable = False
model3.layers[7].trainable = False
model3.summary()
model3.compile(optimizer='adam', loss=lossFunction, metrics=[dice_coef])
history_model3 = model3.fit(gray_x_train, y_train, batch_size=16, nb_epoch=5, verbose=1, shuffle=True, validation_data=(gray_x_val, y_val), callbacks = callbacks)
model3.load_weights('model.h5')
callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]


print("Fine tunning...")
#Fine tuning
#model_end = Model(inputs=[inputs], outputs=[up1])
conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
conv4 = BatchNormalization()(conv4)
up3 = UpSampling2D(size=(2, 2))(conv4) 
up3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
up3 = BatchNormalization()(up3)
up2 = UpSampling2D(size=(2, 2))(up3) 
up2 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
up2 = BatchNormalization()(up2)
up1 = UpSampling2D(size=(2, 2))(up2) 
up1 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1)
up1 = BatchNormalization()(up1)
up0 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
model_end = Model(inputs=[inputs], outputs=[up0])
model_end.layers[1].trainable = True
model_end.layers[3].trainable = True
model_end.layers[5].trainable = True
model_end.layers[7].trainable = True
model_end.compile(optimizer='adam', loss=lossFunction, metrics=[dice_coef])
model_end.summary()
history_model_end = model_end.fit(gray_x_train, y_train, batch_size=16, nb_epoch=100, verbose=1, shuffle=True, validation_data=(gray_x_val, y_val), callbacks = callbacks)
model_end.load_weights('model.h5')


________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

## ***Join both models***

In [0]:
#Join both outputs:

model_binarycrossentropy = keras.models.clone_model(model_end)
#model_dicecoef = keras.models.clone_model(model_end)

In [0]:
#Get the images sets
train_binarycrossentropy = model_binarycrossentropy.predict(gray_x_train)
train_dicecoef = model_dicecoef.predict(gray_x_train)
val_binarycrossentropy = model_binarycrossentropy.predict(gray_x_val)
val_dicecoef = model_dicecoef.predict(gray_x_val)

join_x_train = np.concatenate((train_binarycrossentropy, train_dicecoef), axis=3)
join_x_val = np.concatenate((val_binarycrossentropy, val_dicecoef), axis=3)

In [0]:
#Neural network
callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]

#Choose loss function
lossFunction = 'binary_crossentropy'
#lossFunction = dice_coef_loss

join_model = Sequential()
join_model.add(Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=(image_size, image_size, 2))) 
join_model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

join_model.summary()
  
join_model.compile(optimizer='adam', loss=lossFunction)

history_join_model = join_model.fit(join_x_train, y_train, epochs=50, batch_size=16, validation_data=(join_x_val, y_val), shuffle = True, callbacks = callbacks)
join_model.load_weights('model.h5')


________________________________________________________________________________________________________________________

In [0]:
#Get the images sets
train_binarycrossentropy = model_binarycrossentropy.predict(gray_x_train)
train_dicecoef = model_dicecoef.predict(gray_x_train)
val_binarycrossentropy = model_binarycrossentropy.predict(gray_x_val)
val_dicecoef = model_dicecoef.predict(gray_x_val)

join_x_train = [train_binarycrossentropy, train_dicecoef]
join_x_val = [val_binarycrossentropy, val_dicecoef]

In [0]:
#Neural network
callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]

#Choose loss function
lossFunction = 'binary_crossentropy'
#lossFunction = dice_coef_loss

input1 = Input((128, 128, 1))
input2 = Input((128, 128, 1))

jn = concatenate([input1,input1], axis = 3)
jn = Conv2D(8, (3, 3), activation='relu', padding='same')(jn)
jn = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(jn)
join_model = Model(inputs=[input1, input2], outputs=[jn])
join_model.compile(optimizer='adam', loss=lossFunction)

history_join_model = join_model.fit(join_x_train, y_train, epochs=250, batch_size=16, validation_data=(join_x_val, y_val), shuffle = True, callbacks = callbacks)
join_model.load_weights('model.h5')

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

## ***Skip connection models***

In [0]:
#Choose loss function
#lossFunction = 'binary_crossentropy'
lossFunction = dice_coef_loss

callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]


#New model with skip layers:

inputs = Input((128, 128, 1))
conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
conv1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
pool1 = Dropout(0.6) (pool1)
conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
conv2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
pool2 = Dropout(0.6) (pool2)
conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
conv3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
pool3 = Dropout(0.6) (pool3)

conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
conv4 = BatchNormalization()(conv4)

up3 = UpSampling2D(size=(2, 2))(conv4) 
up3 = concatenate([conv3,up3], axis = 3)
up3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
up3 = BatchNormalization()(up3)
up2 = UpSampling2D(size=(2, 2))(up3) 
up2 = concatenate([conv2,up2], axis = 3)
up2 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
up2 = BatchNormalization()(up2)
up1 = UpSampling2D(size=(2, 2))(up2) 
up1 = concatenate([conv1,up1], axis = 3)
up1 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1)
up1 = BatchNormalization()(up1)
up0 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
model_SC = Model(inputs=[inputs], outputs=[up0])

model_SC.compile(optimizer='sgd', loss=lossFunction)

#model.summary()
#plot_model(model, to_file="concatenate_model_structure.png", show_shapes=True, show_layer_names=True)

history_model_SC = model_SC.fit(gray_x_train, y_train, batch_size=16, nb_epoch=150, verbose=1, shuffle=True, validation_data=(gray_x_val, y_val), callbacks = callbacks)
model_SC.load_weights('model.h5')



________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

## ***UNET architecture***

In [0]:
#Definition of U-NET

def unet_model(lossFunction, pretrained_weights = None,input_size = (128,128,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer='sgd', loss = lossFunction)
    
    #model.summary()

    return model


In [0]:
#Choose loss function
#lossFunction = 'binary_crossentropy'
lossFunction = dice_coef_loss

callbacks = [ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)]
unet = unet_model(lossFunction = lossFunction)

history_unet = unet.fit(gray_x_train, y_train, epochs=150, batch_size=16, validation_data=(gray_x_val, y_val), shuffle = True, callbacks = callbacks)
unet = unet.load_weights('model.h5')


________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________

## ***Download the tests***

In [0]:
#Download the figures file
from google.colab import files

!zip -r /content/figures.zip /content/figures
files.download("/content/figures.zip")